## Question Answering

In [6]:
import pandas as pd
from transformers import pipeline

In [14]:
df = pd.read_parquet('dados/id_desc_clr.parquet')
df.shape

(3452, 6)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3452 entries, 0 to 3615
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                3452 non-null   object
 1   desc              3452 non-null   object
 2   desc_sent         3452 non-null   object
 3   desc_token_clr    3452 non-null   object
 4   desc_bigram_clr   3452 non-null   object
 5   desc_trigram_clr  3452 non-null   object
dtypes: object(6)
memory usage: 188.8+ KB


### Extração do nome das empresas

In [3]:
qa_model = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
def get_company_name(desc):
    question = 'What is the name of the company mentioned in this text?'
    answer = qa_model(question=question, context=desc)
    return answer['answer']

In [17]:
# Teste
desc = df['desc'].iloc[0]
print('Descrição:', desc)
print('Nome da empresa:', get_company_name(desc))

Descrição: Vectra is a cybersecurity platform that uses AI to detect and automatically stop attackers in real-time while collecting data to perform conclusive incident investigations. It specializes in cloud and network detection and response (NDR) – from cloud and data center workloads to accounts, user, and IoT devices. The company offers four applicationson the Cognito platform to address high-priority use cases. Cognito Detect for Office365 and Azure AD finds and stops attacks in enterprise SaaS applications and the Microsoft 365 ecosystem. Cognito Stream sends security-enriched metadata in zeek format to data lakes and SIEMs. Cognito Recall is a cloud-based application to store and investigate threats in enriched metadata. And Cognito Detect uses AI to reveal and prioritize hidden and unknown attackers at speed. Vectra was founded in 2011 and is based in San Jose, California.Read More
Nome da empresa: Vectra


In [19]:
df2 = df[['desc']].head(10)
df2['company_name'] = df2['desc'].apply(get_company_name)

In [20]:
df2

,desc,company_name
0,Vectra is a cybersecurity platform that uses A...,Vectra
1,Roadzen is a global leader in the P&C insuranc...,Roadzen
2,Restream is a multi-streaming solution that al...,Restream
3,DLP works for 1% of companies; fortunately the...,Cyberhaven
4,Shift Technology delivers AI-native decision a...,Shift Technology
5,Clutch is a Toronto-based technology company t...,Clutch
6,"FoodLogiQ specializes in supplier management, ...",FoodLogiQ
7,3DFortify is an advanced manufacturing startup...,3DFortify
8,"gradient A.I., we’re tackling some of the most...",gradient A.I.
9,WebPT is a rehab therapy platform designed fo...,WebPT
